In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#sklearn libraries
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

from catboost import Pool, CatBoostRegressor # pip install catboost

import json
from tqdm import tqdm

### Reformat Core DataFrame

In [2]:
csv = "property_tax_records_2017.csv"
df = pd.read_csv(csv)
df = df[df['GeneralUseType'] == 'Residential']
columns_to_keep = ['CENTER_LAT', 'CENTER_LON', 'ZIPcode5', 'PropertyLocation', 'TotalValue', \
            'SQFTmain', 'Units', 'Bedrooms', 'Bathrooms', 'EffectiveYearBuilt', \
            'SpecificUseDetail1', 'SpecificUseDetail2', 'rowID', 'RecordingDate','TaxRateArea_CITY']
df = df[columns_to_keep]
# Drop the rows where at least one element is missing.
df = df.dropna()

/Users/kx/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# reformat dataframe
df['house_type'] = np.where(df['SpecificUseDetail1']=='Townhouse Format', 'townhouse', 'house')
df.loc[df.SpecificUseDetail2 == 'Condominium', 'house_type'] = 'condo'
df.loc[df.SpecificUseDetail2 == 'Planned Unit Development (PUD)', 'house_type'] = 'pud'
df['pool'] = np.where(df['SpecificUseDetail2'].str.contains('Pool'), 'yes', 'no')
df['RecordingDate'] = pd.to_datetime(df['RecordingDate'], format='%Y%m%d')
df['ZIPcode5'] = df['ZIPcode5'].astype(int).astype(str)
df['coordinates'] = list(zip(df.CENTER_LAT, df.CENTER_LON))
df = df.drop(['SpecificUseDetail1', 'SpecificUseDetail2'], axis=1)
df = df[(df['SQFTmain'] != 0) & (df['Bedrooms'] != 0) & (df['Bathrooms'] != 0)]
df['TotalValue'] = df['TotalValue'].str.strip('$')
df['TotalValue'] = pd.to_numeric(df['TotalValue'])

In [4]:
july = (df['RecordingDate'] >= '2016-07-01') & (df['RecordingDate'] < '2016-08-01')

In [5]:
df_july = df.loc[july]

In [6]:
df_july.head()

,CENTER_LAT,CENTER_LON,ZIPcode5,PropertyLocation,TotalValue,SQFTmain,Units,Bedrooms,Bathrooms,EffectiveYearBuilt,rowID,RecordingDate,TaxRateArea_CITY,house_type,pool,coordinates
12,33.719806,-118.325900,90732,2207 WARMOUTH ST LOS ANGELES CA 90732,1694000.0,2592,1,3,3,1959,20177563037006,2016-07-29,LOS ANGELES,house,no,"(33.7198056, -118.3258995)"
13,33.719599,-118.325547,90732,2197 WARMOUTH ST LOS ANGELES CA 90732,159442.0,2333,1,4,3,1963,20177563037032,2016-07-20,LOS ANGELES,house,no,"(33.71959942, -118.32554666)"
19,33.738143,-118.355705,90275,4319 ADMIRABLE DR RNCHO PALOS VRDS CA 90275,1250000.0,1865,1,3,3,1956,20177564003021,2016-07-08,RANCHO PALOS VERDES,house,yes,"(33.73814294, -118.35570495)"
35,33.735480,-118.351255,90275,4032 EXULTANT DR RNCHO PALOS VRDS CA 90275,1095000.0,2329,1,3,3,1961,20177564009023,2016-07-29,RANCHO PALOS VERDES,house,no,"(33.73548029, -118.35125511)"
42,33.736590,-118.346261,90275,32222 SEA RAVEN DR RNCHO PALOS VRDS CA 90275,132192.0,2030,1,4,2,1963,20177564013020,2016-07-27,RANCHO PALOS VERDES,house,no,"(33.7365904, -118.34626136)"


In [7]:
len(df_july)

16717

In [8]:
df_july.to_csv('property_tax_records_July2016.csv', index = False)

### Integrate Census DataFrame

In [9]:
with open('census_dict.txt') as f:
    census_features = json.load(f)
census_labels = ['zipcode', 'pop_density', 'HH_density', 'Family', 'HH_Kids', 
          'Owner', 'Vacancy', 'Age', 'Race_W', 'Race_B', 'Race_A', 'Race_H', 'Income', 
          'Year_Built', 'Commute', 'Drive', 'Carpool2', 'Carpool3', 'Transit', 'Walk', 'Other']
df_census = pd.DataFrame.from_dict(census_features, orient='index')
df_census.columns = census_labels

In [10]:
df_census = df_census[['zipcode', 'HH_Kids', 'Owner']]

In [11]:
df_july['ZIPcode5'] = df_july['ZIPcode5'].astype(str)
df_census['zipcode'] = df_census['zipcode'].astype(str)
df_integrated = pd.merge(left=df_july,right=df_census, left_on='ZIPcode5', right_on='zipcode')
df_integrated = df_integrated.drop('zipcode', axis=1)

/Users/kx/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Integrate Parcel Area DataFrame

In [12]:
df_parcels = pd.read_csv('df_parcels.csv')
df_parcels.columns = ['rowID', 'lot_area']
df_integrated = pd.merge(left=df_integrated,right=df_parcels, left_on='rowID', right_on='rowID')

### Integrate Great School DataFrame

In [13]:
df_gs = pd.read_csv('df_greatschool.csv')
df_gs = df_gs[['rowID', 'closest_school', 'num_choices', 'closest_rating']]
df_gs.columns = ['rowID', 'closest_school', 'num_school_choices', 'closest_school_rating']
df_integrated = pd.merge(left=df_integrated,right=df_gs, left_on='rowID', right_on='rowID')

### Integrate Elevation

In [14]:
df_elevation = pd.read_csv('df_elevation.csv')
df_elevation = df_elevation[['rowID', 'elevation']]
df_integrated = pd.merge(left=df_integrated,right=df_elevation, left_on='rowID', right_on='rowID')

### Integrate Nearby Places

In [15]:
df_nearby_places = pd.read_csv('df_nearby_places.csv')
df_nearby_places = df_nearby_places[['rowID', 'groceries', 'parks']]
df_integrated = pd.merge(left=df_integrated,right=df_nearby_places, left_on='rowID', right_on='rowID')

In [16]:
len(df_integrated)

16561

### Integrate Crime DataFrame

In [17]:
df_crime = pd.read_csv('df_shooting.csv')
df_crime['zipcode'] = df_crime['zipcode'].astype(str)
df_crime.columns = ['crime', 'zipcode']

In [18]:
df_integrated = pd.merge(left=df_integrated,right=df_crime, left_on='ZIPcode5', right_on='zipcode')
df_integrated = df_integrated.drop('zipcode', axis=1)

In [19]:
len(df_integrated)

16561

### Integrate Walkscore DataFrame

In [20]:
df_walkscore = pd.read_csv('df_walkscore.csv')

In [21]:
df_walkscore = df_walkscore[df_walkscore['walkscore'] != 0][['address','walkscore', 'transit_score', 'bike_score']]

In [22]:
df_integrated = pd.merge(left=df_integrated,right=df_walkscore, left_on='PropertyLocation', right_on='address')
df_integrated = df_integrated.drop('address', axis=1)

In [23]:
df_integrated.shape

(7070, 29)

### Export Integrated DataFrame

In [24]:
df_integrated.head()

,CENTER_LAT,CENTER_LON,ZIPcode5,PropertyLocation,TotalValue,SQFTmain,Units,Bedrooms,Bathrooms,EffectiveYearBuilt,...,closest_school,num_school_choices,closest_school_rating,elevation,groceries,parks,crime,walkscore,transit_score,bike_score
0,33.719806,-118.325900,90732,2207 WARMOUTH ST LOS ANGELES CA 90732,1694000.0,2592,1,3,3,1959,...,0.27,1.0,6.0,49.198009,3.0,4.0,0.0,18,11,9
1,33.719599,-118.325547,90732,2197 WARMOUTH ST LOS ANGELES CA 90732,159442.0,2333,1,4,3,1963,...,0.26,1.0,6.0,48.694706,3.0,4.0,0.0,18,11,9
2,33.738143,-118.355705,90275,4319 ADMIRABLE DR RNCHO PALOS VRDS CA 90275,1250000.0,1865,1,3,3,1956,...,1.27,0.0,9.0,82.350777,0.0,4.0,0.0,4,0,0
3,33.735480,-118.351255,90275,4032 EXULTANT DR RNCHO PALOS VRDS CA 90275,1095000.0,2329,1,3,3,1961,...,1.06,0.0,9.0,95.827736,0.0,4.0,0.0,11,0,0
4,33.736590,-118.346261,90275,32222 SEA RAVEN DR RNCHO PALOS VRDS CA 90275,132192.0,2030,1,4,2,1963,...,0.77,1.0,9.0,135.799332,0.0,4.0,0.0,10,0,0


In [25]:
df_integrated.columns

Index(['CENTER_LAT', 'CENTER_LON', 'ZIPcode5', 'PropertyLocation',
       'TotalValue', 'SQFTmain', 'Units', 'Bedrooms', 'Bathrooms',
       'EffectiveYearBuilt', 'rowID', 'RecordingDate', 'TaxRateArea_CITY',
       'house_type', 'pool', 'coordinates', 'HH_Kids', 'Owner', 'lot_area',
       'closest_school', 'num_school_choices', 'closest_school_rating',
       'elevation', 'groceries', 'parks', 'crime', 'walkscore',
       'transit_score', 'bike_score'],
      dtype='object')

In [27]:
df_integrated = df_integrated[df_integrated['Bedrooms'] < 10] # there are a couple of properties that have hundreds of bedrooms
df_integrated = df_integrated[df_integrated['EffectiveYearBuilt'] != 0] # there are a couple of properties that have no effective year

In [29]:
df_integrated.to_csv('df_integrated_0714.csv', index = False)